In [2]:
import pandas as pd
# import googlemaps
# import json
# import openpyxl
# import time
import requests

In [4]:
import requests
import pandas as pd

headers = {
    "accept": "application/json",
    "Authorization": "Bearer "
}

all_data = []  # Create an empty list to store response data

offset = 0
limit = 50

while offset < 1000:
    url = f"https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit={limit}&offset={offset}"
    print(url)

    response = requests.get(url, headers=headers)
    data = response.json()

    all_data.extend(data['businesses'])  # Append the 'businesses' data to the list

    offset += limit

df = pd.json_normalize(all_data)  # Convert the list of dictionaries to a DataFrame
df.to_excel("yelp_places.xlsx", index=False)

https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=0
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=50
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=100
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=150
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=200
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=250
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=300
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=350
htt

In [94]:
import requests
import pandas as pd

headers = {
    "accept": "application/json",
    "Authorization": "Bearer "
}

all_data = []  # Create an empty list to store response data

offset = 0
limit = 50
data_received = True

while data_received:
    url = f"https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit={limit}&offset={offset}"
    print(url)

    response = requests.get(url, headers=headers)
    data = response.json()

    if len(data['businesses']) == 0:
        data_received = False
    else:
        all_data.extend(data['businesses'])  # Append the 'businesses' data to the list
        offset += limit

df = pd.json_normalize(all_data)  # Convert the list of dictionaries to a DataFrame
df.to_excel("yelp_places_more.xlsx", index=False)

https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=0
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=50
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=100
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=150
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=200
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=250
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=300
https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=350
htt

KeyError: 'businesses'

In [89]:
import requests

url = "https://api.yelp.com/v3/businesses/search"
headers = {
    "accept": "application/json",
    "Authorization": "Bearer "}
params = {
    "location": "Louisville, KY",
    "term": "restaurant",
    "sort_by": "best_match",
    "limit": 50
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    # Request was successful
    data = response.json()
    # Process the data
    df = pd.json_normalize(data['businesses'])
    print(df)
else:
    # Request encountered an error
    if response.status_code == 429:
        print("Rate limit exceeded. Please wait and try again later.")
    else:
        print("An error occurred. Status code:", response.status_code)

                        id                                              alias  \
0   wI0IMYHBXIY6AM6gC7cfwg                             hammerheads-louisville   
1   21Nt4ST1RXsUTjsu9S5U4w                             the-eagle-louisville-2   
2   odteHpNf_EFYEDAos1TKBg                        north-of-bourbon-louisville   
3   IP1u6lnesN-coXw9H0J0dA  doc-crows-southern-smokehouse-and-raw-bar-loui...   
4   9Mn8QELLYEJDr4L_UFP2Lw  copper-and-kings-rooftop-bar-and-restaurant-lo...   
5   rea3CaVeJPwYH_7m2uDjEQ           nami-modern-korean-steakhouse-louisville   
6   tFAlCwUDs3X5MZFbQ8M6LA                                   perso-louisville   
7   cR_EjXJ3FOdrmVewM0vDmA                          toastys-tavern-louisville   
8   MgsUx7fkVk56iW37gxcI2g                       lou-lou-on-market-louisville   
9   fvlJYHAYxBPrUlXR5tiU-A    olivers-chop-house-and-bourbon-bar-louisville-2   
10  h7NwCSS2ukHEhPP8q-fpwA                           ccs-kitchen-louisville-2   
11  5Rpp36FyodXg2jZ8UKgbMA  

In [75]:
import requests
import pandas as pd

headers = {
    "accept": "application/json",
    "Authorization": "Bearer "
}

all_data = []  # Create an empty list to store response data

offset = 0
while offset <= 950:
    url = f"https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset={offset}"
    print(url)

    response = requests.get(url, headers=headers)
    data = response.json()

    all_data.extend(data['businesses'])  # Append the 'businesses' data to the list

    offset += 50

df = pd.json_normalize(all_data)  # Convert the list of dictionaries to a DataFrame
print(df)

https://api.yelp.com/v3/businesses/search?location=Louisville%2C%20KY&term=restaurant&sort_by=best_match&limit=50&offset=0


KeyError: 'businesses'